<a href="https://colab.research.google.com/github/NiluEst/DSSS_Homework9/blob/main/Ex9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Create chatbot with Telegram BotFather

from telegram import Update
from telegram.ext import Application, CommandHandler, MessageHandler, filters


#Use LLM to generate text
import torch
from transformers import pipeline

generator = pipeline('text-generation', model="TinyLlama/TinyLlama-1.1B-Chat-v1.0", torch_dtype=torch.bfloat16, device_map="auto")

async def generate_text(update: Update, context) -> None:
    user_message = update.message.text
    messages = [
        {
            "role": "system",
            "content": "You are a friendly chatbot who always responds in the style of a pirate",
        },
        {"role": "user", "content": "Can you tell me something about squirrels?"},
    ]
    prompt = generator.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    outputs = generator(prompt, max_new_tokens=256, do_sample=True, temperature=0.7, top_k=50, top_p=0.95)
    #do_sample=True sorgt dafür, dass das Modell nicht immer das wahrscheinlichste Wort auswählt, sondern auch mal ein zufälliges
    #temperature=0.7: nahe 0: konservativ, nahe 1: kreativ
    #top_k=50: Anzahl der Wörter, die in die Auswahl einbezogen werden
    #top_p=0.95: Wahrscheinlichkeit, dass ein Wort ausgewählt wird
    print(outputs[0]["generated_text"])

    await update.message.reply_text(outputs[0]["generated_text"])


async def start(update: Update, context) -> None:
    await update.message.reply_text('Hello! I am your bot. How can I help you?')


def main():
    application = Application.builder().token(Telegram_API).build()
    application.add_handler(CommandHandler("start", start))
    application.add_handler(MessageHandler(filters.TEXT & ~filters.COMMAND, generate_text))
    application.run_polling()

if __name__ == '__main__':
    main()


